In [21]:
FONT_LINKS = """
https://web.archive.org/web/20130404225932/http://www.iol.ie/~sob/gadelica.zip
http://web.archive.org/web/20120830224624/http://www.gaelchlo.com/bunardgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20120830224559/http://www.gaelchlo.com/bunartgc.zip
http://web.archive.org/web/20191216105737/http://www.gaelchlo.com/bundgc.zip
http://web.archive.org/web/20120830224343/http://www.gaelchlo.com/bunnod.zip
http://web.archive.org/web/20191218121436/http://www.gaelchlo.com/buntgc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20240119141814/https://www.gaelchlo.com/bungc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20240119143113/https://www.gaelchlo.com/seangc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20240619180031/http://www.gaelchlo.com/urgc.zip
"""

In [22]:
FONTS = [x for x in FONT_LINKS.split("\n") if x != ""]

In [ ]:
for font in FONTS:
  !wget {font}
  !unzip {font.split("/")[-1]}

In [11]:
!mkdir fonts
!mv *.ttf fonts

In [ ]:
!apt install fontforge

In [26]:
!for i in */*otf;do ttf=$(basename "$i" .otf).ttf; echo fontforge -c 'Open("' "$i" '"); Generate("' "fonts/$ttf" '")';done

fontforge -c Open(" bunargc/bunargc.otf "); Generate(" fonts/bunargc.ttf ")
fontforge -c Open(" bungc/bundgc.otf "); Generate(" fonts/bundgc.ttf ")
fontforge -c Open(" bungc/bungc.otf "); Generate(" fonts/bungc.ttf ")
fontforge -c Open(" bungc/bunigc.otf "); Generate(" fonts/bunigc.ttf ")
fontforge -c Open(" bungc/buntgc.otf "); Generate(" fonts/buntgc.ttf ")
fontforge -c Open(" gadelica/Gadelica.otf "); Generate(" fonts/Gadelica.ttf ")
fontforge -c Open(" seangc/seangc.otf "); Generate(" fonts/seangc.ttf ")
fontforge -c Open(" seangc/seantgc.otf "); Generate(" fonts/seantgc.ttf ")
fontforge -c Open(" urgc/urgc.otf "); Generate(" fonts/urgc.ttf ")
fontforge -c Open(" urgc/urigc.otf "); Generate(" fonts/urigc.ttf ")
fontforge -c Open(" urgc/urtgc.otf "); Generate(" fonts/urtgc.ttf ")


In [ ]:
'Open("my.ttf"); Generate("my.otf")'